// Databricks notebook source exported at Mon, 8 Feb 2016 23:29:54 UTC


#![Wikipedia Logo](http://sameerf-dbc-labs.s3-website-us-west-2.amazonaws.com/data/wikipedia/images/w_logo_for_labs.png)

# Explore English Wikipedia clickstream
### Time to complete: 20 minutes

#### Business Questions:

* Question # 1) What are the top 10 articles requested from Wikipedia?
* Question # 2) Who sent the most traffic to Wikipedia in Feb 2015?
* Question # 3) What were the top 5 trending articles on Twitter in Feb 2015?
* Question # 4) What are the most requested missing pages?
* Question # 5) What does the traffic inflow vs outflow look like for the most requested pages?
* Question # 6) What does the traffic flow pattern look like for the San Francisco article? Create a visualization for this.

#### Technical Accomplishments:

* Learn how to use the Spark CSV Library to read structured files
* Explore the Spark UIs to understand the performance characteristics of your Spark jobs
* Mix SQL and DataFrames queries
* Join 2 DataFrames
* Create a Google visualization to understand the clickstream traffic for the 'San Francisco' article
* Bonus: Explain in DataFrames and SQL



Dataset: http://datahub.io/dataset/wikipedia-clickstream/resource/be85cc68-d1e6-4134-804a-fd36b94dbb82

Lab idea from [Ellery Wulczyn](https://ewulczyn.github.io/Wikipedia_Clickstream_Getting_Started/)

 The file we are exploring in this lab is the February 2015 English Wikipedia Clickstream data. 

According to Wikimedia: 

>"The data contains counts of (referer, resource) pairs extracted from the request logs of English Wikipedia. When a client requests a resource by following a link or performing a search, the URI of the webpage that linked to the resource is included with the request in an HTTP header called the "referer". This data captures 22 million (referer, resource) pairs from a total of 3.2 billion requests collected during the month of February 2015."

### DataFrames
A `sqlContext` object is your entry point for working with structured data (rows and columns) in Spark.

Let's use the `sqlContext` to read a table of the Clickstream data.

##### First import stuff

In [1]:
import org.apache.spark.sql.functions._
import sqlContext.implicits._

In [2]:
// Notice that the sqlContext in DSE is actually a HiveContext
sqlContext

org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@2d34a88a

 A `HiveContext` includes additional features like the ability to write queries using the more complete HiveQL parser, access to Hive UDFs, and the ability to read data from Hive tables. In general, you should always aim to use the `HiveContext` over the more limited `sqlContext`.

 First let's load the data into a DataFrame.

 Use the [Spark CSV Library](https://github.com/databricks/spark-csv) to parse the tab separated file:

In [2]:
//Create a DataFrame with the anticipated structure
val clickstreamDF = sqlContext.read.format("com.databricks.spark.csv")
  .option("header", "true")
  .option("delimiter", "\t")
  .option("mode", "PERMISSIVE")
  .option("inferSchema", "true")
  .load("file:///mnt/ephemeral/summitdata/2015_01_clickstream.tsv")

In [3]:
clickstreamDF.show()

+--------+-------+----+--------------------+----------+----+
| prev_id|curr_id|   n|          prev_title|curr_title|type|
+--------+-------+----+--------------------+----------+----+
|    null|3632887| 415|         other-empty|        !!|null|
|    null|3632887| 113|        other-google|        !!|null|
|    null|3632887|  33|     other-wikipedia|        !!|null|
|    null| 600744|  25|         other-yahoo|       !!!|null|
|    null| 600744|1193|        other-google|       !!!|null|
|    null| 600744|1065|         other-empty|       !!!|null|
|25014178| 600744|  44|         Jerry_Fuchs|       !!!|null|
|34552784| 600744|  11|   Flight_Facilities|       !!!|null|
|    4499| 600744|  21|                Band|       !!!|null|
| 1446971| 600744|  14|Gold_Standard_Lab...|       !!!|null|
| 8526330| 600744|  70|          Myth_Takes|       !!!|null|
| 1507641| 600744| 139|     LCD_Soundsystem|       !!!|null|
| 2064029| 600744|  31|             Out_Hud|       !!!|null|
|26780719| 600744|  40|L

 `printSchema()` prints out the schema, the data types and whether a column can be null:

In [5]:
clickstreamDF.printSchema()

root
 |-- prev_id: integer (nullable = true)
 |-- curr_id: integer (nullable = true)
 |-- n: integer (nullable = true)
 |-- prev_title: string (nullable = true)
 |-- curr_title: string (nullable = true)
 |-- type: string (nullable = true)



 The two id columns (prev_id and curr_id) are not used in this lab, so let's create a new DataFrame without them:

In [4]:
val clickstreamDF2 = clickstreamDF.select($"prev_title", $"curr_title", $"n", $"type")

In [7]:
clickstreamDF2.show(5)

+---------------+----------+----+----+
|     prev_title|curr_title|   n|type|
+---------------+----------+----+----+
|    other-empty|        !!| 415|null|
|   other-google|        !!| 113|null|
|other-wikipedia|        !!|  33|null|
|    other-yahoo|       !!!|  25|null|
|   other-google|       !!!|1193|null|
+---------------+----------+----+----+
only showing top 5 rows



 Here is what the 6 columns mean:

- `prev_id`: *(note, we already dropped this)* if the referer does not correspond to an article in the main namespace of English Wikipedia, this value will be empty. Otherwise, it contains the unique MediaWiki page ID of the article corresponding to the referer i.e. the previous article the client was on

- `curr_id`: *(note, we already dropped this)* the MediaWiki unique page ID of the article the client requested

- `prev_title`: the result of mapping the referer URL to the fixed set of values described above

- `curr_title`: the title of the article the client requested

- `n`: the number of occurrences of the (referer, resource) pair

- `type`
  - "link" if the referer and request are both articles and the referer links to the request
  - "redlink" if the referer is an article and links to the request, but the request is not in the production enwiki.page table
  - "other" if the referer and request are both articles but the referer does not link to the request. This can happen when clients search or spoof their refer

 Referers were mapped to a fixed set of values corresponding to internal traffic or external traffic from one of the top 5 global traffic sources to English Wikipedia, based on this scheme:

>- an article in the main namespace of English Wikipedia -> the article title
- any Wikipedia page that is not in the main namespace of English Wikipedia -> `other-wikipedia`
- an empty referer -> `other-empty`
- a page from any other Wikimedia project -> `other-internal`
- Google -> `other-google`
- Yahoo -> `other-yahoo`
- Bing -> `other-bing`
- Facebook -> `other-facebook`
- Twitter -> `other-twitter`
- anything else -> `other-other`

### Reading from disk vs memory

The 1.2 GB Clickstream file is currently on S3, which means each time you scan through it, your Spark cluster has to read the 1.2 GB of data remotely over the network.

 Call the `count()` action to check how many rows are in the DataFrame and to see how long it takes to read the DataFrame from S3:

In [8]:
clickstreamDF2.count()

Long = 21996601

 So it took about 1 minute to read the 1.2 GB file into your Spark cluster. The file has 22.5 million rows/lines. We should cache the DataFrame into memory so it'll be faster to work with:

In [9]:
// cache() is a lazy operation, so we need to call an action (like count) to materialize the cache
clickstreamDF2.cache().count()

Long = 21996601

 How much faster is the DataFrame to read from memory?

In [10]:
clickstreamDF2.count()

Long = 21996601

 Less than a second!

 
### Question #1:
** What are the top 10 articles requested from Wikipedia?**

 We start by grouping by the current title and summing the number of occurrences of the referrer/resource pair:

In [11]:
clickstreamDF2.groupBy("curr_title").sum().limit(10).show()

+--------------------+------+
|          curr_title|sum(n)|
+--------------------+------+
|     Arne_Hamarsland|    14|
|      Arnold_Kramish|    39|
|      Arnold_Mindell|   451|
|Arnold_Williams_(...|    24|
|Arnoldus_Andries_...|    36|
|                Arod|   124|
|             Arruazu|   166|
|      Arsen_Minasian|    33|
|            Artangel|   257|
|      Arthroconidium|   465|
+--------------------+------+



 To see just the top 10 articles requested, we also need to order by the sum of n column, in descending order.

** Challenge 1:** Can you build upon the code in the cell above to also order by the sum column in descending order, then limit the results to the top ten?

In [12]:
//Type in your answer here...
clickstreamDF2.groupBy("curr_title").sum().orderBy($"sum(n)".desc).limit(10).show()

+--------------------+---------+
|          curr_title|   sum(n)|
+--------------------+---------+
|           Main_Page|489603866|
|          Chris_Kyle|  4211238|
|             Malware|  4067814|
|       Charlie_Hebdo|  2581856|
|     Leptin_receptor|  2565856|
|              Chrome|  1792151|
|       Script_kiddie|  1779860|
|American_Sniper_(...|  1753218|
|Winston-Salem/For...|  1542559|
|        Flow_control|  1369143|
+--------------------+---------+



 Spark SQL lets you seemlessly move between DataFrames and SQL. We can run the same query using SQL:

In [5]:
//First register the table, so we can call it from SQL
clickstreamDF2.registerTempTable("clickstream")

 First do a simple "Select all" query from the `clickstream` table to make sure it's working:
 
 #### This is more like the way you would normally use sql in a real application  `%%sql` does something like this

In [18]:
val rows = sqlContext.sql("SELECT * FROM clickstream LIMIT 5")
rows.collect.foreach(println)

[other-empty,!!,415,null]
[other-google,!!,113,null]
[other-wikipedia,!!,33,null]
[other-yahoo,!!!,25,null]
[other-google,!!!,1193,null]


In [6]:
%%sql SELECT * FROM clickstream LIMIT 5

prev_title,curr_title,n,type
other-empty,!!,415,null
other-google,!!,113,null
other-wikipedia,!!,33,null
other-yahoo,!!!,25,null
other-google,!!!,1193,null


 Now we can translate our DataFrames query to SQL:

In [20]:
%%sql SELECT curr_title, SUM(n) AS top_articles
FROM clickstream GROUP BY curr_title
ORDER BY top_articles DESC LIMIT 10

curr_title,top_articles
Main_Page,489603866
Chris_Kyle,4211238
Malware,4067814
Charlie_Hebdo,2581856
Leptin_receptor,2565856
Chrome,1792151
Script_kiddie,1779860
American_Sniper_(film),1753218
Winston-Salem/Forsyth_County_Schools,1542559
Flow_control,1369143


 The most requested articles tend to be about media that was popular in February 2015, with a few exceptions.

 SQL also has some handy commands like `DESC` (describe) to see the schema + data types for the table:

In [23]:
%%sql DESC clickstream

col_name,data_type,comment
prev_title,string,
curr_title,string,
n,int,
type,string,


 You can use the `SHOW FUNCTIONS` command to see what functions are supported by Spark SQL:

In [24]:
%%sql SHOW FUNCTIONS

function
!
!=
%
&
*
+
-
/
<
<=


 `EXPLAIN` can be used to understand the Physical Plan of the SQL statement:

In [25]:
%%sql EXPLAIN 
  SELECT curr_title, SUM(n) AS top_articles
    FROM clickstream GROUP BY curr_title
    ORDER BY top_articles DESC

plan
== Physical Plan ==
"Sort [top_articles#321L DESC], true, 0"
+- ConvertToUnsafe
"+- Exchange rangepartitioning(top_articles#321L DESC,200), None"
+- ConvertToSafe
"+- TungstenAggregate(key=[curr_title#4], functions=[(sum(cast(n#2 as bigint)),mode=Final,isDistinct=false)], output=[curr_title#4,top_articles#321L])"
"+- TungstenExchange hashpartitioning(curr_title#4,200), None"
"+- TungstenAggregate(key=[curr_title#4], functions=[(sum(cast(n#2 as bigint)),mode=Partial,isDistinct=false)], output=[curr_title#4,sum#346L])"
"+- InMemoryColumnarTableScan [curr_title#4,n#2], InMemoryRelation [prev_title#3,curr_title#4,n#2,type#5], true, 10000, StorageLevel(true, true, false, true, 1), ConvertToUnsafe, None"


 Since Spark SQL is not designed to be a low-latency transactional database (like MySQL or Cassandra), INSERTs, UPDATEs and DELETEs are not supported. (Spark SQL is typically used for batch analysis of data)

 
### Question #2:
** Who sent the most traffic to Wikipedia in Feb 2015?** So, who were the top referers to Wikipedia?

In [26]:
clickstreamDF2
  .groupBy("prev_title")
  .sum()
  .orderBy($"sum(n)".desc)
  .show(10)

+---------------+----------+
|     prev_title|    sum(n)|
+---------------+----------+
|   other-google|1583172874|
|    other-empty|1039020465|
|other-wikipedia|  99610051|
|          other|  83225546|
|     other-bing|  64539160|
|    other-yahoo|  49074812|
|      Main_Page|  25077303|
|  other-twitter|  22541757|
| other-facebook|   2599787|
|     Chris_Kyle|   1547364|
+---------------+----------+
only showing top 10 rows



 The top referer by a large margin is Google. Next comes refererless traffic (usually clients using HTTPS). The third largest sender of traffic to English Wikipedia are Wikipedia pages that are not in the main namespace (ns = 0) of English Wikipedia. Learn about the Wikipedia namespaces here:
https://en.wikipedia.org/wiki/Wikipedia:Project_namespace

Also, note that Twitter sends 10x more requests to Wikipedia than Facebook.

 
### Question #3:
** What were the top 5 trending articles on Twitter in Feb 2015?**

** Challenge 2:** Can you answer this question using DataFrames?

In [9]:
//Type in your answer here
clickstreamDF2
  .filter("prev_title = 'other-twitter'")
  .groupBy("curr_title")
  .sum()
  .orderBy($"sum(n)".desc)
  .limit(5).show

+----------------+------+
|      curr_title|sum(n)|
+----------------+------+
| André_the_Giant|286710|
|Harald_Bluetooth|258130|
|    London_Stone|247418|
|         Yaodong|184078|
|    New_Horizons|174157|
+----------------+------+



 ** Challenge 3:** Try re-writing the query above using SQL:

In [10]:
%%sql SELECT curr_title, SUM(n) AS top_twitter FROM clickstream
WHERE prev_title = 'other-twitter'
GROUP BY curr_title
ORDER BY top_twitter DESC LIMIT 5

curr_title,top_twitter
André_the_Giant,286710
Harald_Bluetooth,258130
London_Stone,247418
Yaodong,184078
New_Horizons,174157


 
### Question #4:
** What are the most requested missing pages? ** (These are the articles that someone should create on Wikipedia!)

 The type column of our table has 3 possible values:

In [29]:
sqlContext.sql("SELECT DISTINCT type FROM clickstream").show

+----+
|type|
+----+
|null|
+----+



 These are described as:
  - **link** - if the referer and request are both articles and the referer links to the request
  - **redlink** - if the referer is an article and links to the request, but the request is not in the production enwiki.page table
  - **other** - if the referer and request are both articles but the referer does not link to the request. This can happen when clients search or spoof their refer

 Redlinks are links to a Wikipedia page that does not exist, either because it has been deleted, or because the author is anticipating the creation of the page. Seeing which redlinks are the most viewed is interesting because it gives some indication about demand for missing content.

Let's find the most popular redlinks:

In [30]:
clickstreamDF2.filter("type = 'redlink'").groupBy("curr_title").sum().orderBy($"sum(n)".desc).limit(5).show()

+----------+------+
|curr_title|sum(n)|
+----------+------+
+----------+------+



 Indeed there doesn't appear to be an article on the Russian actress [Anna Lezhneva](https://en.wikipedia.org/wiki/Anna_Lezhneva) on Wikipedia. Maybe you should create it!

Note that if you clicked on the link for Anna Lezhneva in this cell, then you registered another Redlink for her article.

 
### Question #5:
** What does the traffic inflow vs outflow look like for the most requested pages? **

 Wikipedia users get to their desired article by either searching for the article in a search engine or navigating from one Wikipedia article to another by following a link. For example, depending on which technique a user used to get to his desired article of **San Francisco**, the (`prev_title`, `curr_title`) tuples would look like:
- (`other-google`, `San_Francisco`)
or
- (`Berkeley`, `San_Francisco`)

 Lets look at the ratio of incoming to outgoing links for the most requested pages.

 First, find the pageviews per article:

In [7]:
val pageviewsPerArticleDF = clickstreamDF2
  .groupBy("curr_title")
  .sum(
  ).withColumnRenamed("sum(n)", "in_count")


pageviewsPerArticleDF.show(10)

+--------------------+--------+
|          curr_title|in_count|
+--------------------+--------+
|Celaenorrhinus_le...|      16|
|            Cellulin|      22|
|Cementerio_Católi...|      20|
|Centaurs_in_popul...|     392|
|Centennial_Challe...|     389|
|Center_for_the_St...|      85|
|Central_Avenue_Hi...|      33|
|Central_Baptist_T...|     103|
|Central_Counterpa...|    3061|
|Central_High_Scho...|     169|
+--------------------+--------+
only showing top 10 rows



 Above we can see that the `.17_Remington` article on Wikipedia in Feb 2015, got 2,143 views.

 Then, find the link clicks per article:

In [32]:
val linkclicksPerArticleDF = clickstreamDF2
  .groupBy("prev_title")
  .sum()
  .withColumnRenamed("sum(n)", "out_count")


linkclicksPerArticleDF.show(10)

+--------------------+---------+
|          prev_title|out_count|
+--------------------+---------+
|Robins_Air_Force_...|      574|
|     Resident_Evil_4|     9178|
|No_Strings_Attach...|     2230|
|        Tawny_Kitaen|     3783|
| Master_(Doctor_Who)|    15501|
|        Folding@home|     1205|
|    Mersenne_twister|     2236|
|          Plant_cell|     3680|
|Electron_transpor...|     3083|
|           Ecosystem|    11667|
+--------------------+---------+
only showing top 10 rows



 So, when people went to the `David_Janson` article on Wikipedia in Feb 2015, 340 times they clicked on a link in that article to go to a next article. 

 Join the two DataFrames we just created to get a wholistic picture:

In [35]:
val in_outDF = pageviewsPerArticleDF
    .join(linkclicksPerArticleDF, ($"curr_title" === $"prev_title"))
    .orderBy($"in_count".desc)

in_outDF.show(10)

+--------------------+---------+--------------------+---------+
|          curr_title| in_count|          prev_title|out_count|
+--------------------+---------+--------------------+---------+
|           Main_Page|489603866|           Main_Page| 25077303|
|          Chris_Kyle|  4211238|          Chris_Kyle|  1547364|
|             Malware|  4067814|             Malware|    10111|
|       Charlie_Hebdo|  2581856|       Charlie_Hebdo|   413185|
|     Leptin_receptor|  2565856|     Leptin_receptor|      118|
|              Chrome|  1792151|              Chrome|     6096|
|       Script_kiddie|  1779860|       Script_kiddie|     2066|
|American_Sniper_(...|  1753218|American_Sniper_(...|  1085765|
|Winston-Salem/For...|  1542559|Winston-Salem/For...|      188|
|        Flow_control|  1369143|        Flow_control|     1008|
+--------------------+---------+--------------------+---------+
only showing top 10 rows



 The `curr_title` and `prev_title` above are the same, so we can just display one of them in the future. Next, add a new `ratio` column to easily see whether there is more `in_count` or `out_count` for an article:

In [34]:
val in_out_ratioDF = in_outDF.withColumn("ratio", $"out_count" / $"in_count").cache()

in_out_ratioDF.select($"curr_title", $"in_count", $"out_count", $"ratio").show(5)

+---------------+---------+---------+--------------------+
|     curr_title| in_count|out_count|               ratio|
+---------------+---------+---------+--------------------+
|      Main_Page|489603866| 25077303|0.051219577175479244|
|     Chris_Kyle|  4211238|  1547364|   0.367436843987445|
|        Malware|  4067814|    10111|0.002485610207349697|
|  Charlie_Hebdo|  2581856|   413185| 0.16003409950051437|
|Leptin_receptor|  2565856|      118|4.598855118915481E-5|
+---------------+---------+---------+--------------------+
only showing top 5 rows



 We can see above that when clients went to the **Alive** article, almost nobody clicked any links in the article to go on to another article.

But 49% of people who visited the **Fifty Shades of Grey** article clicked on a link in the article and continued to browse Wikipedia.

 
### Question #6:
** What does the traffic flow pattern look like for the "San Francisco" article? Create a visualization for this. **

In [36]:
in_out_ratioDF.filter("curr_title = 'San_Francisco'").show()

+-------------+--------+-------------+---------+------------------+
|   curr_title|in_count|   prev_title|out_count|             ratio|
+-------------+--------+-------------+---------+------------------+
|San_Francisco|  146200|San_Francisco|    45930|0.3141586867305062|
+-------------+--------+-------------+---------+------------------+



 Hmm, so about 41% of clients who visit the San_Francisco page, click on through to another article.

 Which referrers send the most traffic to the "San Francisco" article?

In [8]:
%%sql SELECT * FROM clickstream
    WHERE curr_title LIKE 'San_Francisco'
    ORDER BY n DESC LIMIT 10

prev_title,curr_title,n,type
other-google,San_Francisco,56493,null
other-empty,San_Francisco,50011,null
other-wikipedia,San_Francisco,7645,null
other,San_Francisco,2718,null
other-bing,San_Francisco,2291,null
Main_Page,San_Francisco,2272,null
other-yahoo,San_Francisco,1587,null
Pornhub,San_Francisco,1247,null
San_Francisco_Bay_Area,San_Francisco,1170,null
List_of_United_States_cities_by_population,San_Francisco,1065,null


 Here's the same query using DataFrames and `show()`:

In [38]:
clickstreamDF2.filter($"curr_title".rlike("""^San_Francisco$""")).orderBy($"n".desc).show(10)

+--------------------+-------------+-----+----+
|          prev_title|   curr_title|    n|type|
+--------------------+-------------+-----+----+
|        other-google|San_Francisco|56493|null|
|         other-empty|San_Francisco|50011|null|
|     other-wikipedia|San_Francisco| 7645|null|
|               other|San_Francisco| 2718|null|
|          other-bing|San_Francisco| 2291|null|
|           Main_Page|San_Francisco| 2272|null|
|         other-yahoo|San_Francisco| 1587|null|
|             Pornhub|San_Francisco| 1247|null|
|San_Francisco_Bay...|San_Francisco| 1170|null|
|List_of_United_St...|San_Francisco| 1065|null|
+--------------------+-------------+-----+----+
only showing top 10 rows



 ** Challenge 4:** Which future articles does the San_Francisco article send most traffic onward to? Try writing this query using the DataFrames API:

In [39]:
//Type in your answer here...
clickstreamDF2.filter($"prev_title".rlike("""^San_Francisco$""")).orderBy($"n".desc).show()

+-------------+--------------------+----+----+
|   prev_title|          curr_title|   n|type|
+-------------+--------------------+----+----+
|San_Francisco|List_of_people_fr...|1600|null|
|San_Francisco|  Golden_Gate_Bridge|1472|null|
|San_Francisco|          California|1152|null|
|San_Francisco|         Los_Angeles| 913|null|
|San_Francisco|Transamerica_Pyramid| 840|null|
|San_Francisco|1906_San_Francisc...| 833|null|
|San_Francisco|Neighborhoods_in_...| 824|null|
|San_Francisco| Ed_Lee_(politician)| 750|null|
|San_Francisco|     Alcatraz_Island| 727|null|
|San_Francisco|San_Jose,_California| 651|null|
|San_Francisco| Northern_California| 632|null|
|San_Francisco|San_Francisco_Bay...| 596|null|
|San_Francisco|San_Francisco_(di...| 569|null|
|San_Francisco|           San_Diego| 553|null|
|San_Francisco|Chinatown,_San_Fr...| 552|null|
|San_Francisco|San_Francisco_Int...| 520|null|
|San_Francisco|Lombard_Street_(S...| 512|null|
|San_Francisco|List_of_United_St...| 482|null|
|San_Francisc

 Above we can see the topics most people are interested in, when they get to the San_Francisco article. The [Golden_Gate_Bridge](https://en.wikipedia.org/wiki/Golden_Gate_Bridge) is the second most clicked on link in the San_Francisco article.

 Finally, we'll use a Google Visualization library to create a Sankey diagram. Sankey diagrams are a flow diagram, in which the width of the arrows are shown proportionally to the flow quantify traffic:

 The chart above shows how people get to a Wikipedia article and what articles they click on next.

This diagram shows incoming and outgoing traffic to the "San Francisco" article. We can see that most people found the "San Francisco" page through Google search and only a small fraction of the readers went on to another article (most went on to the "List of people in San Francisco" article)

 Note that it is also possible to programmatically add in the values in the HTML, so you don't have to hand-code it. But to keep things simple, we've hand coded it above.

 
### Bonus:
** Learning about Explain to understand Catalyst internals **

 The `explain()` method can be called on a DataFrame to understand its physical plan:

In [40]:
in_out_ratioDF.explain()

== Physical Plan ==
InMemoryColumnarTableScan [curr_title#4,in_count#465L,prev_title#3,out_count#490L,ratio#556], InMemoryRelation [curr_title#4,in_count#465L,prev_title#3,out_count#490L,ratio#556], true, 10000, StorageLevel(true, true, false, true, 1), Sort [in_count#465L DESC], true, 0, None


 You can also pass in `true` to see the logical & physical plans:

In [41]:
in_out_ratioDF.explain(true)

== Parsed Logical Plan ==
'Project [*,('out_count / 'in_count) AS ratio#556]
+- Sort [in_count#465L DESC], true
   +- Join Inner, Some((curr_title#4 = prev_title#3))
      :- Project [curr_title#4,sum(n)#464L AS in_count#465L]
      :  +- Aggregate [curr_title#4], [curr_title#4,(sum(cast(n#2 as bigint)),mode=Complete,isDistinct=false) AS sum(n)#464L]
      :     +- Project [prev_title#3,curr_title#4,n#2,type#5]
      :        +- Relation[prev_id#0,curr_id#1,n#2,prev_title#3,curr_title#4,type#5] CsvRelation(<function0>,Some(file:///mnt/ephemeral/summitdata/2015_01_clickstream.tsv),true,	,",null,#,PERMISSIVE,COMMONS,false,false,false,null,true,null)
      +- Project [prev_title#3,sum(n)#489L AS out_count#490L]
         +- Aggregate [prev_title#3], [prev_title#3,(sum(cast(n#2 as bigint)),mode=Complete,isDistinct=false) AS sum(n)#489L]
            +- Project [prev_title#3,curr_title#4,n#2,type#5]
               +- Relation[prev_id#0,curr_id#1,n#2,prev_title#3,curr_title#4,type#5] CsvRelati

 This concludes the Clickstream lab.

 
### Homework:
** Recreate the same visualization above, but instead of the "San Francisco" article, choose another topic you care about (maybe `Apache_Spark`?).**

 ###Post lab demos below:

This section will be covered by the instructor using a hands-on demo shortly...

 Which pages multiplied input clicks the most?

In [ ]:
clickstreamDF2.show(3)

In [ ]:
val inClicksDF = clickstreamDF2
  .groupBy($"curr_title")
  .sum()
  .withColumnRenamed("sum(n)", "in_clicks")
  .select($"curr_title", $"in_clicks")
  .as("in")

In [ ]:
inClicksDF.show(5)

In [ ]:
val outClicksDF = clickstreamDF2
	.filter($"type" === "link")
	.groupBy($"prev_title")
	.sum()
	.withColumnRenamed("sum(n)", "out_clicks")
	.select($"prev_title", $"out_clicks")
	.as("out")

In [ ]:
outClicksDF.show(5)

 Notes...Mention that the filter is good, removes 50% of data
Then Aggregate? partial aggregation, dont? worry about details
then shuffle
then reduce side aggregation
Cartesian Product: very expensive? joining every single row on left to rows on right? takes forever
Get?s triggered by UDF.. during join, we?re passing in an arbitrary function.. so the user can do whatever they want in the function?
but take a closer look at the body of the UDF above (title1.toLowercase == title2.toLowerCase, you can see that we?re just doing a simple eqality check
b/c it?s an arbitrary UDF, Catalyst (SQL query optimizer) doesn?t know how to optimize this join b/c it doesn?t understand it.. it?s opaque
Solution: a better UDF, here we explicitly use the built in Spark SQL equality expression that Catalyst does understand. I?m still using a UDF to convert to lowercase, just not for equality
Now verify CP is gone in UI.

In [ ]:
// Define a UDF for comparing article titles
val compareUDF = udf((title1: String, title2: String) => title1.toLowerCase == title2.toLowerCase)

In [ ]:

// STEVE: This join will take long - don't run it. 40000 tasks. And will OOM

// Join these 2 DFs to find the (output clicks) / (input clicks) factor
val joinedDF = outClicksDF
	.join(inClicksDF, compareUDF($"in.curr_title", $"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

joinedDF.orderBy($"multiplication_factor".desc).show()

In [ ]:
// Define a UDF for comparing article titles
val formatUDF = udf((title1: String) => title1.toLowerCase)

In [ ]:
// Fixed performance issue
val joinedDF = outClicksDF
	.join(inClicksDF, formatUDF($"in.curr_title") === formatUDF($"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

joinedDF.orderBy($"multiplication_factor".desc).show()

In [ ]:
// Fixed performance issue
val joinedDF2 = outClicksDF
	.join(inClicksDF, formatUDF($"in.curr_title") === formatUDF($"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

joinedDF2.orderBy($"multiplication_factor").show()

 Interesting, looks like mostly cities, tech and colors.